In [31]:
import pandas as pd
import numpy as np
import joblib  # For loading saved models
from xgboost import XGBRegressor

# Load the trained model
model = XGBRegressor()
model.load_model("xgboost_model.json")  # Load the saved XGBoost model

# Function to take user input
def get_user_input():
    print("\nEnter Match Details for Prediction:")
    
    batting_team = input("Enter Batting Team: ")
    bowling_team = input("Enter Bowling Team: ")
    city = input("Enter City: ")
    overs = float(input("Enter Overs Completed: "))
    balls_left = (20 - overs) * 6
    wickets = int(input("Enter Wickets Fallen: "))
    total_target = int(input("Enter Target Runs (if chasing, else 0): "))
    runs_scored = int(input("Enter Runs Scored So Far: "))

    # Calculate additional features
    runs_left = max(0, total_target - runs_scored) if total_target > 0 else 0
    crr = runs_scored / overs if overs > 0 else 0
    rrr = (runs_left / (balls_left / 6)) if balls_left > 0 else 0

    # Create input DataFrame
    data = pd.DataFrame({
        'match_id':0,
        'batting_team': [batting_team],
        'bowling_team': [bowling_team],
        'city': [city],
        'runs_left': [runs_left],
        'balls_left': [balls_left],
        'wickets_in_hand': [10 - wickets],
        'CRR': [crr],
        'RRR': [rrr]
    })

    return data

# Function to preprocess data
def preprocess_data(data):
    # Load encoders used during training
    teams=pd.read_csv('team_mapping.csv')
    teams_dict = teams.set_index(teams.columns[0])[teams.columns[1]].to_dict()
    cities=pd.read_csv('cities_mapping.csv')
    cities_dict = cities.set_index(cities.columns[0])[cities.columns[1]].to_dict()
    
    # Encode categorical features
    data['batting_team'] = data['batting_team'].map(teams_dict)
    data['bowling_team'] = data['bowling_team'].map(teams_dict)
    data['city'] = data['city'].map(cities_dict)

    return data

# Get user input
user_data = get_user_input()

# Preprocess user input
processed_data = preprocess_data(user_data)

# Predict match score
predicted_score = model.predict(processed_data)

# Show prediction
print(f"\n🏏 Predicted Final Score: {int(predicted_score[0])} Runs")


Enter Match Details for Prediction:
Enter Batting Team: Punjab Kings
Enter Bowling Team: Gujarat Titans
Enter City: Ahmedabad
Enter Overs Completed: 14
Enter Wickets Fallen: 6
Enter Target Runs (if chasing, else 0): 150
Enter Runs Scored So Far: 80

🏏 Predicted Final Score: 161 Runs
